In [1]:
import pandas as pd
import unicodedata
import re

# Funções utilitárias
def normalize(name):
    if not isinstance(name, str): return ""
    name = name.strip().lower()
    return unicodedata.normalize('NFKD', name).encode('ASCII', 'ignore').decode('utf-8')

def clean_team_name(name):
    corrections = {
        "criciumaec": "Criciúma",
        "rodBull": "Bragantino",
        "rodBull]": "Bragantino",
        "athletico": "Athletico-PR",
        "atletico-go": "Atlético-GO",
        "atletico-mg": "Atlético-MG",
        "america-mg": "América-MG",
        "cuiaba": "Cuiabá",
        "gremio": "Grêmio",
        "goias": "Goiás",
        "coritiba": "Coritiba",
        "flamengo": "Flamengo",
        "fluminense": "Fluminense",
        "fortaleza": "Fortaleza",
        "botafogo": "Botafogo",
        "santos": "Santos",
        "palmeiras": "Palmeiras",
        "bahia": "Bahia",
        "cruzeiro": "Cruzeiro",
        "internacional": "Internacional",
        "corinthians": "Corinthians",
        "vasco": "Vasco",
        "bragantino": "Bragantino"
    }
    norm = normalize(name)
    return corrections.get(norm, name.strip())

def clean_placar(p):
    if not isinstance(p, str): return ""
    match = re.match(r"\s*(\d+)\s*x\s*(\d+)\s*", p.lower())
    if match:
        return f"{match.group(1)} x {match.group(2)}"
    return ""

# Carregar arquivos
df1 = pd.read_csv("partidas_corrigidas.csv")
df2 = pd.read_csv("partidas_corrigidas4k.csv")

# Fundir: nomes de df1, placar de df2
merged_df = pd.DataFrame({
    "rodada": df1["rodada"],
    "mandante": df1["mandante"].apply(clean_team_name),
    "visitante": df1["visitante"].apply(clean_team_name),
    "placar": df2["placar"].apply(clean_placar)
})

# Exportar CSV final
merged_df.to_csv("partidas_fundidas_corrigidas.csv", index=False)
print("✅ Arquivo 'partidas_fundidas_corrigidas.csv' criado com sucesso!")


✅ Arquivo 'partidas_fundidas_corrigidas.csv' criado com sucesso!


Observações em relação aos jogos adiados

In [2]:
import pandas as pd

# Carregue seu arquivo CSV (substitua pelo nome correto do arquivo)
df = pd.read_csv("partidas_fundidas_corrigidas.csv")

# Conta quantas linhas têm "Postponed" como mandante ou visitante
postponed_count = (
    df["mandante"].astype(str).str.lower().eq("postponed") |
    df["visitante"].astype(str).str.lower().eq("postponed")
).sum()

print(f"Total de partidas com 'Postponed': {postponed_count}")


Total de partidas com 'Postponed': 8


Identificar os jogos e mudar manualmente

In [3]:
import pandas as pd

# Substitua com o nome do seu arquivo CSV
df = pd.read_csv("partidas_fundidas_corrigidas.csv")

# Normalizar para facilitar comparação
df["mandante"] = df["mandante"].astype(str).str.strip()
df["visitante"] = df["visitante"].astype(str).str.strip()

# Filtrar partidas adiadas
adiados = df[
    (df["mandante"].str.lower() == "postponed") |
    (df["visitante"].str.lower() == "postponed")
]

# Criar uma coluna mais legível
def resumo(row):
    if row["mandante"].lower() == "postponed":
        return f"Rodada {row['rodada']}: {row['visitante']} (mandante adiado)"
    elif row["visitante"].lower() == "postponed":
        return f"Rodada {row['rodada']}: {row['mandante']} (visitante adiado)"
    else:
        return "Erro"

adiados["descricao"] = adiados.apply(resumo, axis=1)

# Mostrar os jogos adiados por rodada
print("\n📌 Jogos Adiados:")
for linha in adiados["descricao"]:
    print(linha)



📌 Jogos Adiados:
Rodada 16: Bragantino (mandante adiado)
Rodada 16: Euon (visitante adiado)
Rodada 17: Fluminense (mandante adiado)
Rodada 17: Internacional (mandante adiado)
Rodada 19: RccuBul (visitante adiado)
Rodada 19: Internacional (mandante adiado)
Rodada 19: Vasco (mandante adiado)
Rodada 2: Euon (visitante adiado)


C:\Users\msdof\AppData\Local\Temp\ipykernel_17056\25773759.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adiados["descricao"] = adiados.apply(resumo, axis=1)
